In [1]:
from pyspark.sql import SparkSession, functions as F, Window

In [2]:
spark = SparkSession \
            .builder \
            .appName("advent-of-code-2021") \
            .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/02 11:40:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [58]:
df_test = spark.read.text("test-input.txt")
df = spark.read.text("input.txt")

In [64]:
# Part 1
def part_1(df):
    return df \
        .withColumn("direction", F.split(df.value, " ").getItem(0)) \
        .withColumn("distance", F.split(df.value, " ").getItem(1).cast("int")) \
        .withColumn("dx", F.when(F.col("direction") == "forward", F.col("distance")).otherwise(0)) \
        .withColumn("dy", F.when(F.col("direction") == "down", F.col("distance"))
                           .when(F.col("direction") == "up", -1 * F.col("distance"))
                           .otherwise(0)) \
        .agg(
            F.sum(F.col("dx")).alias("x"),
            F.sum(F.col("dy")).alias("y")
        ) \
        .withColumn("solution", F.col("x") * F.col("y")) \
        .collect()[0][2]

assert part_1(df_test) == 150

print(f"Solution: {part_1(df)}")

Solution: 1484118


In [67]:
# Part 2
def part_2(df):
    return df \
        .withColumn("direction", F.split(df.value, " ").getItem(0)) \
        .withColumn("distance", F.split(df.value, " ").getItem(1).cast("int")) \
        .withColumn("dx", F.when(F.col("direction") == "forward", F.col("distance")).otherwise(0)) \
        .withColumn("daim", F.when(F.col("direction") == "down", F.col("distance"))
                             .when(F.col("direction") == "up", -1 * F.col("distance"))
                             .otherwise(0)
        ) \
        .withColumn("idx", F.row_number().over(Window.orderBy(F.lit(1)))) \
        .withColumn("aim", F.sum(F.col("daim")).over(Window.orderBy(F.col("idx")))) \
        .withColumn("dy", F.when(F.col("direction") == "forward", F.col("aim") * F.col("distance")).otherwise(0)) \
        .agg(
            F.sum(F.col("dx")).alias("x"),
            F.sum(F.col("dy")).alias("y")
        ) \
        .withColumn("solution", F.col("x") * F.col("y")) \
        .collect()[0][2]

assert part_2(df_test) == 900

print(f"Solution: {part_2(df)}")

21/12/02 14:40:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/02 14:40:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/02 14:40:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/02 14:40:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/02 14:40:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/02 14:40:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/02 1

Solution: 1463827010
